In [ ]:
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Paths
ROOT = Path.cwd().parent
PROCESSED = ROOT / "data" / "processed"
FIGURES = ROOT / "data" / "outputs" / "figures"

# Load results
with open(PROCESSED / "scenario_results.json", "r") as f:
    results = json.load(f)

print("✅ Dados carregados")

## 1. Premissas dos Cenários

In [ ]:
# Tabela de premissas
scenarios = results["scenarios"]

premissas_data = []
for key in ["pessimistic", "base", "optimistic"]:
    s = scenarios[key]
    a = s["assumptions"]
    premissas_data.append({
        "Cenário": s["scenario_name"],
        "Brent (US$/bbl)": f"${a['brent_price']:.0f}",
        "Produção (MMboe/d)": f"{a['production']:.1f}",
        "Δ Reservas (%)": f"{a['reserves_change']*100:+.0f}%",
        "ERP (%)": f"{a['erp']*100:.1f}%",
        "Descrição": s["description"]
    })

df_premissas = pd.DataFrame(premissas_data)
df_premissas

## 2. Resultados por Cenário

In [ ]:
# Tabela de resultados
resultados_data = []
for key in ["pessimistic", "base", "optimistic"]:
    s = scenarios[key]
    resultados_data.append({
        "Cenário": s["scenario_name"],
        "Ke (%)": f"{s['capm']['ke']*100:.2f}%",
        "Fair Value (R$)": f"R$ {s['valuation']['fair_value']:.2f}",
        "Upside (%)": f"{s['valuation']['upside_percent']:+.1f}%",
        "Score Q-VAL": f"{s['qval']['score_final']:.1f}",
        "Recomendação": s['qval']['recommendation']
    })

df_resultados = pd.DataFrame(resultados_data)
df_resultados

## 3. Faixas de Sensibilidade

In [ ]:
# Faixas de sensibilidade
sens = results["sensitivity"]

print("=" * 60)
print("FAIXAS DE SENSIBILIDADE — PETR4")
print("=" * 60)
print(f"\n📊 Score Q-VAL:")
print(f"   Mínimo (Pessimista): {sens['score_range']['min']:.1f}")
print(f"   Máximo (Otimista):   {sens['score_range']['max']:.1f}")
print(f"   Amplitude:           {sens['score_range']['max'] - sens['score_range']['min']:.1f}")

print(f"\n💰 Fair Value:")
print(f"   Mínimo (Pessimista): R$ {sens['fair_value_range']['min']:.2f}")
print(f"   Máximo (Otimista):   R$ {sens['fair_value_range']['max']:.2f}")
print(f"   Amplitude:           R$ {sens['fair_value_range']['max'] - sens['fair_value_range']['min']:.2f}")

print(f"\n📈 Upside:")
print(f"   Mínimo (Pessimista): {sens['upside_range']['min']:+.1f}%")
print(f"   Máximo (Otimista):   {sens['upside_range']['max']:+.1f}%")

## 4. Figuras de Análise de Cenários

In [ ]:
from IPython.display import Image, display
import subprocess

# Converter PDF para PNG para visualização
figures = [
    ("Tornado Chart - Sensibilidade", "sensibilidade_score.pdf"),
    ("Comparativo de Cenários", "cenarios_comparativo.pdf"),
    ("Waterfall - Variação Score", "cenarios_waterfall.pdf")
]

for title, filename in figures:
    pdf_path = FIGURES / filename
    print(f"\n📊 {title}")
    print(f"   Arquivo: {pdf_path}")
    if pdf_path.exists():
        print(f"   ✅ Disponível")
    else:
        print(f"   ❌ Não encontrado")

## 5. Análise de Impacto por Variável

In [ ]:
# Análise de impacto
sens_vars = results["sensitivity"]["variables"]

print("=" * 60)
print("IMPACTO DAS VARIÁVEIS NO SCORE Q-VAL")
print("=" * 60)

# Ordenar por impacto total
sorted_vars = sorted(sens_vars.items(), 
                     key=lambda x: x[1]['upside_impact'] - x[1]['downside_impact'],
                     reverse=True)

for var_name, impacts in sorted_vars:
    total_range = impacts['upside_impact'] - impacts['downside_impact']
    print(f"\n{var_name}:")
    print(f"   📈 Otimista: {impacts['upside_impact']:+.1f} pts")
    print(f"   📉 Pessimista: {impacts['downside_impact']:+.1f} pts")
    print(f"   ↔️  Amplitude: {total_range:.1f} pts")

## 6. Conclusões

In [ ]:
# Conclusões
base = scenarios["base"]
opt = scenarios["optimistic"]
pess = scenarios["pessimistic"]

print("=" * 60)
print("CONCLUSÕES DA ANÁLISE DE CENÁRIOS")
print("=" * 60)

print(f"\n🎯 CENÁRIO BASE:")
print(f"   Score: {base['qval']['score_final']:.1f}/100")
print(f"   Fair Value: R$ {base['valuation']['fair_value']:.2f}")
print(f"   Recomendação: {base['qval']['recommendation']}")

print(f"\n🚀 CENÁRIO OTIMISTA:")
print(f"   Condições: Margem Equatorial + Brent US$90")
print(f"   Score: {opt['qval']['score_final']:.1f}/100 ({opt['qval']['score_final'] - base['qval']['score_final']:+.1f})")
print(f"   Fair Value: R$ {opt['valuation']['fair_value']:.2f} ({opt['valuation']['fair_value']/base['valuation']['fair_value']*100-100:+.1f}%)")
print(f"   Recomendação: {opt['qval']['recommendation']}")

print(f"\n⚠️ CENÁRIO PESSIMISTA:")
print(f"   Condições: Bloqueio Margem + Brent US$50")
print(f"   Score: {pess['qval']['score_final']:.1f}/100 ({pess['qval']['score_final'] - base['qval']['score_final']:+.1f})")
print(f"   Fair Value: R$ {pess['valuation']['fair_value']:.2f} ({pess['valuation']['fair_value']/base['valuation']['fair_value']*100-100:+.1f}%)")
print(f"   Recomendação: {pess['qval']['recommendation']}")

print("\n" + "=" * 60)
print("A análise de cenários revela alta sensibilidade do Score Q-VAL")
print(f"às variáveis Brent e Produção, com amplitude de {sens['score_range']['max'] - sens['score_range']['min']:.1f} pontos.")
print("=" * 60)